Copyright © 2022 LMU Munich Media Informatics Group. All rights reserved.

Based on discussions between [Changkun Ou](https://changkun.de), [Francesco Chiossi](https://www.um.informatik.uni-muenchen.de/personen/mitarbeiter/chiossi/index.html), and [Sven Mayer](https://sven-mayer.com).

Use of this source code is governed by a GNU GPLv3 license that
can be found in the LICENSE file.

In [1]:
import numpy as np
import pandas as pd
import neurokit2 as nk

from dataloader import Dataset, Participant

## EDA Preprocessing

In [ ]:
def sample_rate(arr: np.array) -> float:
    return len(arr) / (np.max(arr) - np.min(arr))*1000

def process_eda(p: Participant, method: str, tide: str, lap: int) -> pd.DataFrame:
    # https://edaguidelines.github.io/analysis/pre
    df = p.eda(method, tide, lap)
    if len(df) < 1:
        print(f'p{p.user_id} {method} {tide} lap{lap} is empty!')
        return
    rate = sample_rate(df['unixTime'].values)

    fall = p.fall(method, tide, lap)
    fall_starts = fall['timestamp_start'].values
    amplitude_prefall = 0
    amplitude_postfall = 0
    peaks_prefall = 0
    peaks_postfall = 0

    dur = 5000  # 5s pre- post-fall
    for tt in fall_starts:
        eda_prefall = df[df['unixTime'].between(tt-dur, tt)]
        if len(eda_prefall) > 15:
            s, _ = nk.eda_process(eda_prefall['EDA'], sampling_rate=rate)
            amplitude_prefall += np.mean(s['SCR_Amplitude'])
            peaks_prefall += len(nk.eda_findpeaks(
                s['EDA_Phasic'], sampling_rate=rate))

        eda_postfall = df[df['unixTime'].between(tt, tt+dur)]
        if len(eda_postfall) > 15:
            s, _ = nk.eda_process(eda_postfall['EDA'], sampling_rate=rate)
            amplitude_postfall += np.mean(s['SCR_Amplitude'])
            peaks_postfall += len(nk.eda_findpeaks(
                s['EDA_Phasic'], sampling_rate=rate))

    eda_clean, _ = nk.eda_process(df['EDA'], sampling_rate=rate)
    return pd.DataFrame([{
        'user_id': p.user_id,
        'method': method,
        'tide': tide,
        'lap': lap,
        'tonic_avg': np.mean(eda_clean['EDA_Tonic']),
        'amplitude_avg': np.mean(eda_clean["SCR_Amplitude"]),
        'amplitude_prefall': amplitude_prefall,
        'amplitude_postfall': amplitude_postfall,
        'peaks_num': np.sum(eda_clean['SCR_Peaks']),
        'peaks_prefall': peaks_prefall,
        'peaks_postfall': peaks_postfall,
        'falls_num': len(fall),
    }])

In [ ]:
df = pd.DataFrame(columns=[
    'user_id',
    'method',
    'tide',
    'lap',
    'tonic_avg',
    'amplitude_avg',
    'amplitude_prefall',
    'amplitude_postfall',
    'peaks_num',
    'peaks_prefall',
    'peaks_postfall',
    'falls_num',
])
d = Dataset()
for uid in d.user_ids:
    p = d.load_participant(uid)
    for m in d.methods:
        for t in d.tides:
            for i in range(4):
                df_p = process_eda(p, m, t, i)
                df = pd.concat([df, df_p], ignore_index=True)
    print(f'{p.user_id} is done')
df.to_csv('clean/eda.csv', index=False)

## ECG Preprocessing

In [ ]:
def process_ecg(p: Participant, method: str, tide: str, lap: int) -> pd.DataFrame:
    df = p.ecg(method, tide, lap)
    if len(df) < 1:
        print(f'p{p.user_id} {method} {tide} lap{lap} is empty!')
        return

    ecg = df['ECG']
    # H10 Technical Specification
    # Sample rate = 130 Hz ± 2 % (Tamb = +20 … +40 °C)
    #               130 Hz ± 5 % (Tamb = -20 … +70 °C)
    # https://github.com/polarofficial/polar-ble-sdk/blob/master/technical_documentation/H10_ECG_Explained.docx
    s, _ = nk.ecg_process(ecg, sampling_rate=130)
    peaks, _ = nk.ecg_peaks(nk.ecg_clean(ecg, sampling_rate=130),
                            sampling_rate=130,
                            correct_artifacts=True)

    try:
        hrv_rmssd = np.mean(nk.hrv(peaks, sampling_rate=130)['HRV_RMSSD'])
    except Exception:
        hrv_rmssd = np.NaN

    df = pd.DataFrame.from_dict([{
        'user_id': p.user_id,
        'method': method,
        'tide': tide,
        'lap': lap,
        # https://neuropsychology.github.io/NeuroKit/functions/ecg.html?highlight=ecg_rate
        'ecg_rate': np.mean(s['ECG_Rate']),
        # https://neuropsychology.github.io/NeuroKit/examples/ecg_hrv/ecg_hrv.html?highlight=rmssd
        'hrv_rmssd': hrv_rmssd,
        # https://neuropsychology.github.io/NeuroKit/functions/hrv.html?highlight=vlf
        'falls_num': len(p.fall(method, tide, lap)),
    }])
    return df

In [ ]:
d = Dataset()
df = pd.DataFrame(columns=[
    'user_id',
    'method',
    'tide',
    'lap',
    'ecg_rate',
    'hrv_rmssd',
    'falls_num',
])

for k in d.user_ids:
    p = d.load_participant(k)
    for m in d.methods:
        for _, t in enumerate(d.tides):
            for i in range(4):
                df = pd.concat([df, process_ecg(p, m, t, i)],
                                ignore_index=True)
df.to_csv('./clean/ecg.csv', index=False)

## Motion Preprocessing

In [3]:
from scipy import signal
import math
from datetime import datetime, timedelta

In [ ]:
d = Dataset()
df = pd.DataFrame(columns=[
    'user_id',
    'method',
    'tide',
    'lap',
    'falls_num',
    'duration',
    'mean_drift_head',
    'std_drift_head',
    'mean_drift_pelvis',
    'std_drift_pelvis',
    'mean_alpha',
    'std_alpha',
    'mean_beta',
    'std_beta',
])
for pid in d.user_ids:
    for m in d.methods:
        for t in d.tides:
            for l in range(4):
                motion = d[pid-10].motion(m, t, l)
                sample_rate = 200
                beam_start = [motion['beam_s_posX'].mean(),
                                motion['beam_s_posZ'].mean()]
                beam_end = [motion['beam_e_posX'].mean(
                ), motion['beam_e_posZ'].mean()]
                # compute difference between two unix timestamp
                print(motion['timestamp'].max(), motion['timestamp'].min())
                if math.isnan(motion['timestamp'].max()) or math.isnan(motion['timestamp'].min()):
                    timediff = 0
                else:
                    timediff = (datetime.fromtimestamp(motion['timestamp'].max()/1000) -
                                datetime.fromtimestamp(
                        motion['timestamp'].min()/1000)).total_seconds()

                # we need swap the position for starting position and ending position.
                if l == 1 or l == 3:
                    beam_start, beam_end = beam_end, beam_start

                if len(motion['head_posX']) > 0 and len(motion['head_posX']) > 0 and len(motion['pelvis_posX']) > 0 and len(motion['pelvis_posZ']) > 0:
                    # only X and Z?
                    head_posX = signal.resample(
                        motion['head_posX'], sample_rate)
                    head_posY = signal.resample(
                        motion['head_posY'], sample_rate)
                    head_posZ = signal.resample(
                        motion['head_posZ'], sample_rate)
                    pelvisX = signal.resample(
                        motion['pelvis_posX'], sample_rate)
                    pelvisY = signal.resample(
                        motion['pelvis_posY'], sample_rate)
                    pelvisZ = signal.resample(
                        motion['pelvis_posZ'], sample_rate)
                    centerX = [(1 - i/sample_rate) * beam_start[0] +
                                (i/sample_rate)*beam_end[0] for i in range(0, sample_rate)]
                    centerZ = [(1 - i/sample_rate) * beam_start[1] +
                                (i/sample_rate)*beam_end[1] for i in range(0, sample_rate)]

                    drift_head = [math.sqrt((head_posX[i]-centerX[i])*(head_posX[i]-centerX[i]) + (
                        head_posZ[i]-centerZ[i])*(head_posZ[i]-centerZ[i])) for i in range(0, sample_rate)]

                    drift_pelvis = [math.sqrt((pelvisX[i]-centerX[i])*(pelvisX[i]-centerX[i]) + (
                        pelvisZ[i]-centerZ[i])*(pelvisZ[i]-centerZ[i])) for i in range(0, sample_rate)]

                    handl_posX = signal.resample(
                        motion['hand_l_posX'], sample_rate)
                    handl_posY = signal.resample(
                        motion['hand_l_posY'], sample_rate)
                    handl_posZ = signal.resample(
                        motion['hand_l_posZ'], sample_rate)
                    handr_posX = signal.resample(
                        motion['hand_r_posX'], sample_rate)
                    handr_posY = signal.resample(
                        motion['hand_r_posY'], sample_rate)
                    handr_posZ = signal.resample(
                        motion['hand_r_posZ'], sample_rate)

                    aX = handl_posX - head_posX
                    aY = handl_posY - head_posY
                    aZ = handl_posZ - head_posZ
                    bX = handr_posX - head_posX
                    bY = handr_posY - head_posY
                    bZ = handr_posZ - head_posZ
                    lenA = np.sqrt(aX * aX + aY * aY + aZ * aZ)
                    lenB = np.sqrt(bX * bX + bY * bY + bZ * bZ)
                    # cosine of angle between two vectors
                    alpha = (aX * bX + aY * bY + aZ * bZ) / (lenA * lenB)
                    alpha = np.arccos(alpha)
                    cX = head_posX - pelvisX
                    cY = head_posY - pelvisY
                    cZ = head_posZ - pelvisZ
                    dX = np.zeros(len(cX))
                    dY = np.ones(len(cY))
                    dZ = np.zeros(len(cZ))
                    lenC = np.sqrt(cX * cX + cY * cY + cZ * cZ)
                    lenD = np.sqrt(dX * dX + dY * dY + dZ * dZ)
                    beta = (cX * dX + cY * dY + cZ * dZ) / (lenC * lenD)
                    beta = np.arccos(beta)

                # for idx, a in enumerate(alpha):
                df = pd.concat([df, pd.DataFrame.from_records([{
                    'user_id': pid,
                    'method': m,
                    'tide': t,
                    'lap': l,
                    'falls_num': len(d[pid-10].fall(m, t, l)),
                    'duration': timediff,
                    'mean_drift_head': np.mean(drift_head),
                    'std_drift_head': np.std(drift_head),
                    'mean_drift_pelvis': np.mean(drift_pelvis),
                    'std_drift_pelvis': np.std(drift_pelvis),
                    'mean_alpha': np.mean(alpha),
                    'std_alpha': np.std(alpha),
                    'mean_beta': np.mean(beta),
                    'std_beta': np.std(beta),
                }])], ignore_index=True)
                print(f'p{pid}-{m}-{t}-lap{l} is done')
df.to_csv(f'clean/motion.csv')

In [4]:
d = Dataset()
df = pd.DataFrame(columns=[
    'user_id',
    'method',
    'tide',
    'lap',
    'falls_num',
    'duration',
    'drift_head',
    'drift_pelvis',
    'alpha',
    'beta',
])
for pid in d.user_ids:
    for m in d.methods:
        for t in d.tides:
            for l in range(4):
                motion = d[pid-10].motion(m, t, l)
                sample_rate = 500
                beam_start = [motion['beam_s_posX'].mean(),
                                motion['beam_s_posZ'].mean()]
                beam_end = [motion['beam_e_posX'].mean(
                ), motion['beam_e_posZ'].mean()]
                # compute difference between two unix timestamp
                print(motion['timestamp'].max(), motion['timestamp'].min())
                if math.isnan(motion['timestamp'].max()) or math.isnan(motion['timestamp'].min()):
                    timediff = 0
                else:
                    timediff = (datetime.fromtimestamp(motion['timestamp'].max()/1000) -
                                datetime.fromtimestamp(
                        motion['timestamp'].min()/1000)).total_seconds()

                # we need swap the position for starting position and ending position.
                if l == 1 or l == 3:
                    beam_start, beam_end = beam_end, beam_start

                if len(motion['head_posX']) > 0 and len(motion['head_posX']) > 0 and len(motion['pelvis_posX']) > 0 and len(motion['pelvis_posZ']) > 0:
                    # only X and Z?
                    head_posX = signal.resample(
                        motion['head_posX'], sample_rate)
                    head_posY = signal.resample(
                        motion['head_posY'], sample_rate)
                    head_posZ = signal.resample(
                        motion['head_posZ'], sample_rate)
                    pelvisX = signal.resample(
                        motion['pelvis_posX'], sample_rate)
                    pelvisY = signal.resample(
                        motion['pelvis_posY'], sample_rate)
                    pelvisZ = signal.resample(
                        motion['pelvis_posZ'], sample_rate)
                    centerX = [(1 - i/sample_rate) * beam_start[0] +
                                (i/sample_rate)*beam_end[0] for i in range(0, sample_rate)]
                    centerZ = [(1 - i/sample_rate) * beam_start[1] +
                                (i/sample_rate)*beam_end[1] for i in range(0, sample_rate)]

                    drift_head = [math.sqrt((head_posX[i]-centerX[i])*(head_posX[i]-centerX[i]) + (
                        head_posZ[i]-centerZ[i])*(head_posZ[i]-centerZ[i])) for i in range(0, sample_rate)]

                    drift_pelvis = [math.sqrt((pelvisX[i]-centerX[i])*(pelvisX[i]-centerX[i]) + (
                        pelvisZ[i]-centerZ[i])*(pelvisZ[i]-centerZ[i])) for i in range(0, sample_rate)]

                    handl_posX = signal.resample(
                        motion['hand_l_posX'], sample_rate)
                    handl_posY = signal.resample(
                        motion['hand_l_posY'], sample_rate)
                    handl_posZ = signal.resample(
                        motion['hand_l_posZ'], sample_rate)
                    handr_posX = signal.resample(
                        motion['hand_r_posX'], sample_rate)
                    handr_posY = signal.resample(
                        motion['hand_r_posY'], sample_rate)
                    handr_posZ = signal.resample(
                        motion['hand_r_posZ'], sample_rate)

                    aX = handl_posX - head_posX
                    aY = handl_posY - head_posY
                    aZ = handl_posZ - head_posZ
                    bX = handr_posX - head_posX
                    bY = handr_posY - head_posY
                    bZ = handr_posZ - head_posZ
                    lenA = np.sqrt(aX * aX + aY * aY + aZ * aZ)
                    lenB = np.sqrt(bX * bX + bY * bY + bZ * bZ)
                    # cosine of angle between two vectors
                    alpha = (aX * bX + aY * bY + aZ * bZ) / (lenA * lenB)
                    alpha = np.arccos(alpha)
                    cX = head_posX - pelvisX
                    cY = head_posY - pelvisY
                    cZ = head_posZ - pelvisZ
                    dX = np.zeros(len(cX))
                    dY = np.ones(len(cY))
                    dZ = np.zeros(len(cZ))
                    lenC = np.sqrt(cX * cX + cY * cY + cZ * cZ)
                    lenD = np.sqrt(dX * dX + dY * dY + dZ * dZ)
                    beta = (cX * dX + cY * dY + cZ * dZ) / (lenC * lenD)
                    beta = np.arccos(beta)

                for idx, a in enumerate(alpha):
                    df = pd.concat([df, pd.DataFrame.from_records([{
                        'user_id': pid,
                        'method': m,
                        'tide': t,
                        'lap': l,
                        'falls_num': len(d[pid-10].fall(m, t, l)),
                        'duration': timediff,
                        'drift_head': drift_head[idx],
                        'drift_pelvis': drift_pelvis[idx],
                        'alpha': a,
                        'beta': beta[idx],
                    }])], ignore_index=True)

                print(f'p{pid}-{m}-{t}-lap{l} is done')

df.to_csv(f'clean/motion_series_percent.csv')

1643275351656 1643275331762
p10-NoInstructions-LowTide-lap0 is done
1643275376702 1643275359232
p10-NoInstructions-LowTide-lap1 is done
1643275398357 1643275377375
p10-NoInstructions-LowTide-lap2 is done
1643275416859 1643275403062
p10-NoInstructions-LowTide-lap3 is done
1643273711850 1643273672479
p10-NoInstructions-HighTide-lap0 is done
1643273752443 1643273719947
p10-NoInstructions-HighTide-lap1 is done
1643273779286 1643273760370
p10-NoInstructions-HighTide-lap2 is done
1643273801083 1643273779409
p10-NoInstructions-HighTide-lap3 is done
1643275813582 1643275796983
p10-Imitation-LowTide-lap0 is done
1643275847676 1643275818431
p10-Imitation-LowTide-lap1 is done
1643275880646 1643275853150
p10-Imitation-LowTide-lap2 is done
1643275926129 1643275881370
p10-Imitation-LowTide-lap3 is done
1643274232836 1643274190467
p10-Imitation-HighTide-lap0 is done
1643274269629 1643274233959
p10-Imitation-HighTide-lap1 is done
1643274302624 1643274270179
p10-Imitation-HighTide-lap2 is done
16432743

## Questionnaires (IPQ, PACES, NASA-TLX) Preprocessing

In [ ]:
d = Dataset()
df = pd.DataFrame(columns=['user_id', 'tide', 'ipq'])
for i in d.user_ids:
    p = d.load_participant(i)
    for c in ['LH', 'HL']:
        scores, _ = p.ipq(c)
        # SP2, INV3, and REAL1 are in inversed order.
        # See:
        # - http://www.igroup.org/pq/ipq/download.php
        # - http://www.igroup.org/pq/ipq/data.php
        reversed = [2, 8, 10]
        for j in reversed:
            scores[j] = 8-scores[j]
        for s in scores:
            df = pd.concat([df, pd.DataFrame.from_records([{
                'user_id': i,
                'tide': 'LowTide' if c == 'LH' else 'HighTide',
                'ipq': s,
            }])], ignore_index=True)
df.to_csv('./clean/ipq.csv', index=False)

In [ ]:
d = Dataset()
df = pd.DataFrame(columns=['user_id', 'method', 'tide', 'tlx'])
for i in d.user_ids:
    p = d.load_participant(i)
    methods = ['NoInstructions', 'Imitation', 'Gamification']
    for t in ['LH', 'HL']:
        for m in methods:
            scores = p.paces(t, m)
            # Calibrate the scores. See:
            #
            # Kendzierski, Deborah, and Kenneth J. DeCarlo. "Physical activity enjoyment scale: Two validation studies." Journal of sport & exercise psychology 13.1 (1991).
            reversed = [0, 3, 4, 6, 8, 9, 10, 12, 13, 15, 16]
            for j in reversed:
                scores[j] = 8 - scores[j]
            for s in scores:
                df = pd.concat([df, pd.DataFrame.from_records([{
                    'user_id': i,
                    'method': m,
                    'tide': 'LowTide' if t == 'LH' else 'HighTide',
                    'tlx': s,
                }])], ignore_index=True)
df.to_csv('clean/paces.csv', index=False)

In [ ]:
d = Dataset()
df = pd.DataFrame(columns=['user_id', 'method', 'tide', 'tlx'])
for uid in d.user_ids:
    p = d.load_participant(uid)
    for t in ['LH', 'HL']:
        for m in d.methods:
            scores = p.nasa_tlx(t, m)
            for s in scores:
                df = pd.concat([df, pd.DataFrame.from_records([{
                    'user_id': uid,
                    'method': m,
                    'tide': 'LowTide' if t == 'LH' else 'HighTide',
                    'tlx': s*5,
                }])], ignore_index=True)
df.to_csv('clean/nasa_tlx.csv', index=False)